In [16]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [18]:
import pickle
with open('poetry_corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)

In [21]:
poems_in_lines = [poem.splitlines() for poem in corpus]

In [23]:
corpus_in_lines = [line for poem in poems_in_lines for line in poem]

In [24]:
corpus_in_lines[0]

'Dear Writers, I’m compiling the first in what I hope is a series of publications I’m calling artists among artists. The theme for issue 1 is “Faggot Dinosaur.” I hope to hear from you! Thank you and best wishes.'

In [ ]:
import string, re
from nltk.corpus import words
import multiprocessing as mp
def clean_text(text):
    if len(sent_tokenize(text)) > 1:
        return ''
    text = text.lower()
    text = text.replace('\%','')
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join(filter(lambda x:x[0]!="@", text.split()))
    return text

with mp.Pool(processes=mp.cpu_count()-1) as pool:
    clean_corpus = pool.map(clean_text, corpus_in_lines)
# p = Process(target=clean_text, args=)
# clean_corpus = Parallel()(delayed(clean_text)(line) for line in corpus_in_lines)
# clean_corpus = list(map(clean_text, corpus_in_lines))

In [ ]:
clean_corpus_lines = [line for line in clean_corpus if len(word_tokenize(line)) > 0]

In [27]:
print(clean_corpus_lines[0])

i think that i shall never fear


In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

tokenizer = Tokenizer(num_words=5)
tokenizer.fit_on_texts(clean_corpus_lines)
# input_sequences = tokenizer.texts_to_sequences(corpus_lines)
total_words = len(tokenizer.word_index) + 1
print(total_words)
input_sequences = []
for line in corpus_lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

139400


In [37]:
import pandas as pd
word_df = pd.DataFrame.from_dict(tokenizer.word_index, orient='index', columns=['count'])

In [39]:
print(word_df.head())
print(word_df.tail())

     count
the      1
and      2
of       3
a        4
to       5
                 count
protoplanetary  139395
moogoogaipan    139396
nottooold       139397
gooing          139398
morninga        139399
